<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Capter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ライブラリインストール
# 各ライブラリの働きはREADME参照
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0
import torch
from transformers import BertJapaneseTokenizer, BertModel


# トークナイザの定義
# トークナイザ:文章をトークン分割して、BERTに入力可能なようにする
# 東北大学が作った事前学習ずみBERTモデルをまず用意
# それを用いて学習済みトークナイザをロード（BertJapaneseTokenizerはおそらくライブラリtransformersに含まれている）
# (このモデルはMeCabを用いて単語分割し、WordPieceを用いてその単語をトークンに分割する。wordPieceの語彙はtokenizer.vocabからアクセス)
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)


# まずは文章をトークン化してみよう
tokenizer.tokenize('明日は自然言語処理の勉強をしよう')




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 8.2 MB/s 
     |████████████████████████████████| 486 kB 69.4 MB/s 
     |████████████████████████████████| 13.4 MB 45.8 MB/s 
     |████████████████████████████████| 880 kB 57.0 MB/s 
     |████████████████████████████████| 3.3 MB 50.7 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=84f6980349589613a734fc677303303c3df2e68fb422ba151a5f38480cc895ac
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=503fcde86060c129955889cbde134fe4d4fb0e95be6ca3849e473941aea1a669
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built ipadic sacremoses


Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う']

In [3]:
# トークナイザの定義
# トークナイザ:文章をトークン分割して、BERTに入力可能なようにする
# 東北大学が作った事前学習ずみBERTモデルをまず用意
# それを用いて学習済みトークナイザをロード（BertJapaneseTokenizerはおそらくライブラリtransformersに含まれている）
# (このモデルはMeCabを用いて単語分割し、WordPieceを用いてその単語をトークンに分割する。wordPieceの語彙はtokenizer.vocabからアクセス)
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [4]:
# まずは文章をトークン化してみよう
tokenizer.tokenize('明日は自然言語処理の勉強をしよう')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う']

In [5]:
# マシンラーニングなどの複合語は、先頭ではないサブワードには＃＃がつく
# サブワード分割はWordPieceのお仕事
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う']

In [6]:
# 未知語を表すトークン：[UNK]
tokenizer.tokenize('機械学習を中国語にすると机器学习という')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'という']

In [7]:
# トークン化が完了したので、次は得られたトークンをトークンIDに変換しよう（BERTには数値で入力）
# この作業を符号化(encode)と呼ぶ

# encode()は、tokenize()の結果をIDに変換したものの先頭に[CLS] (文章の先頭のトークン)、最後に[SEP] (文章終わりのトークン)をおく


input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう')
print(tokenizer.tokenize('明日は自然言語処理の勉強をしよう'))
print(input_ids)

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う']
[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 3]


In [8]:
# 逆変換（ID→トークン）はconvert_ids_to_tokensで行う
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '[SEP]']

In [9]:
# BERTでは通常、複数の文章をまとめて処理する -> 文章のトークン列の長さ（系列長）を揃えないとダメ
# 文章が短ければ、特殊トークン[PAD]を追加する
# 文章が長ければ、末尾のトークンではみ出したものは削除


# 例えばこれを符号化すると、以下のようになる
text = "明日の天気は晴れだ。"

# textを符号化している
# 系列長は12(特殊トークン含む)であり、足りなければ[PAD], 多すぎれば切り捨て(truncation)
encoding = tokenizer(
    text, max_length=12, padding='max_length', truncation=True
)


# input_ids : トークンの符号化結果
# token_type_ids' ： 2つの文章を同時に入力した場合、どちらの文章に属しているのかを判断する
# attention_mask : [PAD]の位置で0, それ以外で1. PADはattentionでは考慮しなくて良い
print('# encoding : ')
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding['input_ids']))



# encoding : 
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [10]:
# max_lengthを短くすると、文章が削られる

# textを符号化している
# 系列長は6(特殊トークン含む)であり、足りなければ[PAD], 多すぎれば切り捨て(truncation)
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=True
)


# input_ids : トークンの符号化結果
# token_type_ids' ： 2つの文章を同時に入力した場合、どちらの文章に属しているのかを判断する
# attention_mask : [PAD]の位置で0, それ以外で1. PADはattentionでは考慮しなくて良い
print('# encoding : ')
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding['input_ids']))

# encoding : 
{'input_ids': [2, 11475, 5, 11385, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


In [11]:
# 複数文章をまとめて符号化

text_list = ['明日の天気は晴れだ。', 'パソコンが急に動かなくなった。']

# 系列長は10(特殊トークン含む)であり、足りなければ[PAD], 多すぎれば切り捨て(truncation)
encoding = tokenizer(
    text_list, max_length=10, padding='max_length', truncation=True
)


# input_ids : トークンの符号化結果
# token_type_ids' ： 2つの文章を同時に入力した場合、どちらの文章に属しているのかを判断する
# attention_mask : [PAD]の位置で0, それ以外で1. PADはattentionでは考慮しなくて良い
print('# encoding : ')
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][1]))

# encoding : 
{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]']
['[CLS]', 'パソコン', 'が', '急', 'に', '動か', 'なく', 'なっ', 'た', '[SEP]']


In [12]:
# tokenizerのpaddingを「longest」に設定すると
# 最長の文章の長さに系列長を合わせる

encoding = tokenizer(
    text_list, padding='longest'
)

print('# encoding : ')
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][1]))


# encoding : 
{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]']
['[CLS]', 'パソコン', 'が', '急', 'に', '動か', 'なく', 'なっ', 'た', '。', '[SEP]']


In [13]:
# transformers(ライブラリ)の提供するBERTに値を入力する際には、データ型をtorch.Tensorにしておく必要がある
# tokenizerの引数に「return_tensors='pt'」と付け加えることでデータ型変更可能

encoding = tokenizer(
    text_list, padding='longest', return_tensors='pt'
)

print(encoding)

{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0,
             0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     8,
             3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [14]:
# ここから、実際にBERTモデルの使い方を学ぶ
# transformersのバートモデルをロード
bert = BertModel.from_pretrained(model_name)

# bertをGPUに載せる
bert = bert.cuda()

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [15]:
# モデル概要
# num_hidden_layers : レイヤー数
# hidden_size : BERTの出力次元数
# max_position_embeddings : 最大で入力できるトークン列の長さ

print(bert.config)

BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



In [18]:
# バッチサイズ：まとめて処理する文章の数

encoding = tokenizer(
    text_list,
    max_length = 32,
    padding = 'max_length',
    truncation=True,
    return_tensors='pt'
)

# データをGPUに載せる
# encoding.items()はdict型 -> key valueの組み合わせ
# keyをk, valueをv.cuda()としてGPUに突っ込んでるんだと思う

print(encoding.items())
encoding = { k : v.cuda() for k, v in encoding.items() }
print(encoding)

# bertで処理
# 辞書型encodingの中身を展開して関数に入力

# output = bert(
#     input_ids=encoding['input_ids'],
#     attention_mask=encoding['attention_mask'],
#     token_type_ids=encoding['token_type_ids']
# )

# 上記コードの省略形
output = bert(**encoding)
# 最終層の出力
last_hidden_state = output.last_hidden_state

dict_items([('input_ids', tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     8,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])), ('token_type_ids', tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])), ('attention_mask', tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 

In [20]:
# 出力テンソルのサイズ
# [バッチサイズ, 系列長, 隠れ状態の次元数]
print(last_hidden_state.size())

# i番目の文章に含まれるj番目のトークンの分散表現
# last_hidden_state[i, j]
print(last_hidden_state[1, 1])

torch.Size([2, 32, 768])
tensor([ 2.1030e-01,  4.3554e-01, -3.3010e-02,  1.1276e-01,  1.7338e-01,
        -3.8727e-01, -1.4225e-01, -2.1831e-02,  7.0929e-01,  1.2820e-01,
        -1.7813e-01,  3.8195e-01,  1.8803e-01, -5.0586e-01, -2.4604e-02,
         6.6034e-01, -3.6920e-01, -1.0582e-01,  3.5404e-01, -2.4468e-01,
        -3.7272e-01, -4.5828e-01,  3.4351e-01, -4.1613e-01, -3.4940e-01,
        -2.4177e-01, -3.1484e-01, -6.0683e-01,  7.6463e-02,  5.7444e-01,
        -2.0198e-01, -2.7453e-01,  1.5271e-01, -1.9025e-01, -1.1113e-01,
         9.9064e-03,  7.8439e-02, -5.6840e-02, -3.4166e-01, -1.9815e-01,
        -3.8176e-02,  3.9712e-01,  1.1200e-01,  9.8846e-01,  1.3539e-01,
         2.5618e-01,  2.1362e-01, -2.0378e-01, -1.7304e-01, -8.3455e-02,
         6.7937e-01,  2.3632e-01,  6.0747e-01,  3.3469e-01,  6.0564e-02,
         3.3551e-02,  3.0007e-02, -1.0812e-01,  1.8126e-01,  3.3926e-01,
        -3.4851e-01, -4.1329e-01,  5.7043e-01, -6.3324e-01, -3.0123e-01,
        -8.5524e-02, -1.74

In [21]:
# torch.no_grad() : 計算の途中経過を保存しないことで、メモリと計算時間を減らす

with torch.no_grad():
  output = bert(**encoding)
  last_hidden_state = output.last_hidden_state

In [ ]:
# last_hidden_stateをCPUに移す（今はGPU上にある）
# last_hidden_state　= last_hidden_state.cpu()

# last_hidden_stateをnumpy.ndarrayに変換
# last_hidden_state = last_hidden_state.numpy()

# last_hidden_stateをリストに変換
# last_hidden_state = last_hidden_state.tolist()